In [ ]:
user=st.session_state['username']
data_rep=data_rep[data_rep["ID_Superviseur"]==user]

In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import folium


In [5]:
#data=pd.read_excel("Data_Simulation.xlsx", sheet_name="Data")
geoData=gpd.read_file("Cameroun.shp")

c:\Users\LK\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Cameroun.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [10]:
data["Région"].unique()

array(['Adamaoua', 'Centre', 'Est', 'Extrême-Nord', 'Littoral', 'Nord',
       'Nord-Ouest', 'Ouest', 'Sud', 'Sud-Ouest'], dtype=object)

In [6]:
geoData["Nom_Régio"]

0        ADAMAOUA
1          CENTRE
2             EST
3    EXTREME-NORD
4        LITTORAL
5            NORD
6      NORD-OUEST
7           OUEST
8             SUD
9       SUD-OUEST
Name: Nom_Régio, dtype: object

In [7]:
# Define mapping dictionary for region names
region_mapping = {
    'ADAMAOUA': 'Adamaoua',
    'CENTRE': 'Centre',
    'EST': 'Est',
    'EXTREME-NORD': 'Extrême-Nord',
    'LITTORAL': 'Littoral',
    'NORD': 'Nord',
    'NORD-OUEST': 'Nord-Ouest',
    'OUEST': 'Ouest',
    'SUD': 'Sud',
    'SUD-OUEST': 'Sud-Ouest'
}

# Replace the values in geoData
geoData['Nom_Régio'] = geoData['Nom_Régio'].replace(region_mapping)

In [8]:
geoData=geoData[["Nom_Régio","Chef_Lieu", "geometry"]]
geoData.columns=["Région","Chef_Lieu","geometry"]

In [14]:
geoData = geoData.rename(columns={"Chef_Lieu": "Chef_Lieu"})
geoData = geoData.rename(columns={"Nom_Régio": "Région"})

In [9]:
geo_data_survey=geoData.merge(data, on="Région", how="inner")

In [10]:
geo_data_survey=gpd.GeoDataFrame(geo_data_survey, geometry="geometry")

In [11]:
geo_data_survey.to_file("geo_data_survey.shp")

C:\Users\LK\AppData\Local\Temp\ipykernel_13600\2161536805.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geo_data_survey.to_file("geo_data_survey.shp")
c:\Users\LK\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'ID_Enquêteur' to 'ID_Enquêt'
  ogr_write(
c:\Users\LK\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'ID_Contrôleur' to 'ID_Contrô'
  ogr_write(
c:\Users\LK\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'ID_Superviseur' to 'ID_Supervi'
  ogr_write(
c:\Users\LK\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Type_Questionnaire' to 'Type_Quest'
  ogr_write(
c:\Users\LK\AppData\Local\Programs\Python\Python

In [12]:
geo_df=gpd.read_file("geo_data_survey.shp")

In [ ]:
# Count questionnaires by region
region_counts = geo_df.groupby('Région').size().reset_index(name='count')

# Dissolve and fix geometries
choropleth_data = geo_df.copy()
choropleth_data['geometry'] = choropleth_data['geometry'].buffer(0)  # Fix invalid geometries
choropleth_data = choropleth_data.dissolve(by='Région')
choropleth_data = choropleth_data.merge(region_counts, on='Région')

# Create the choropleth map
fig = px.choropleth(choropleth_data,
                    geojson=choropleth_data.geometry,
                    locations=choropleth_data.index,
                    color='count',
                    title='Nombre de questionnaires par région',
                    labels={'count': 'Nombre de questionnaires'},
                    color_continuous_scale='Viridis')

# Update layout
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})

fig.show()

In [11]:
from IPython.display import IFrame

# Convert date columns to string before visualization
geo_df['Date_Colle'] = geo_df['Date_Colle'].astype(str)

# Count questionnaires by region and merge with geo_df
region_counts = geo_df.groupby('Région').size().reset_index(name='count')
geo_df = geo_df.merge(region_counts, on='Région', how='left')

# Créer une carte centrée sur le Cameroun
m = folium.Map(location=[7.3697, 12.3547], zoom_start=6)

# Définir l'échelle de couleur du rouge au vert
colormap = folium.LinearColormap(
    colors=['red', 'yellow', 'green'],
    vmin=region_counts['count'].min(),
    vmax=region_counts['count'].max(),
    caption='Nombre de questionnaires'
)

# Ajouter le choropleth à la carte
folium.Choropleth(
    geo_data=geo_df.__geo_interface__,  # Convert to geo interface
    data=region_counts,
    columns=['Région', 'count'],
    key_on='feature.properties.Région',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Nombre de questionnaires par région',
).add_to(m)

# Ajouter les labels avec le nombre de questionnaires
style_function = lambda x: {'fillColor': '#ffffff', 
                          'color':'#000000', 
                          'fillOpacity': 0.1, 
                          'weight': 0.1}

highlight_function = lambda x: {'fillColor': '#000000', 
                              'color':'#000000', 
                              'fillOpacity': 0.50, 
                              'weight': 0.1}

NIL = folium.features.GeoJson(
    geo_df,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Région', 'count'],
        aliases=['Région:', 'Nombre de questionnaires:'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)

m.add_child(NIL)
m.add_child(colormap)

# Afficher la carte
m

# Save and display the map
m.save('cameroon_map.html')
IFrame(src='cameroon_map.html', width='100%', height=600)

In [ ]:
def make_school_map(geo_df, style_carte="OpenStreetMap"):
    """
    Crée une carte interactive montrant la distribution des questionnaires par région
    et les emplacements des établissements.
    
    Parameters:
    -----------
    df : GeoDataFrame
        DataFrame contenant les données géospatiales des questionnaires
    style_carte : str, default="OpenStreetMap"
        Style de fond de carte Folium (options: "OpenStreetMap", "cartodbpositron", "cartodbdark_matter", etc.)
    
    Returns:
    --------
    folium.Map
        Carte interactive avec la distribution des questionnaires
    """
    
    center_lat, center_lon = 7.3697, 12.3547
    center_lat, center_lon = 7.3697, 12.3547
    
    # Créer la carte
    m = folium.Map(location=[center_lat, center_lon], 
                   zoom_start=6,
                   tiles=style_carte)
    
    # Préparer les données pour la choroplèthe
    region_counts = geo_df.groupby('Région').size().reset_index(name='count')
    
    # Calculer les coordonnées moyennes pour chaque région
    region_coords = geo_df.groupby('Région').agg({
        'Latitude': 'mean',
        'Longitude': 'mean'
    }).reset_index()
    
    # Fusionner les comptages avec les coordonnées
    region_data = region_counts.merge(region_coords, on='Région')
    
    # Créer l'échelle de couleur
    colormap = folium.LinearColormap(
        colors=['#fee5d9', '#a50f15'],
        vmin=region_counts['count'].min(),
        vmax=region_counts['count'].max(),
        caption='Nombre de questionnaires par région'
    )
    
    # Ajouter la choroplèthe
    folium.Choropleth(
        geo_data=geo_df.__geo_interface__,
        data=region_counts,
        columns=['Région', 'count'],
        key_on='feature.properties.Région',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Nombre de questionnaires par région'
    ).add_to(m)
    
    # Ajouter les popup d'information pour chaque région
    NIL = folium.features.GeoJson(
        geo_df,
        style_function=lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1},
        control=False,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['Région', 'count'],
            aliases=['Région:', 'Nombre de questionnaires:'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
        )
    )
    
    # Créer un groupe pour les établissements
    school_group = folium.FeatureGroup(name='Établissements')
    
    # Ajouter les marqueurs pour chaque établissement unique
    schools_data = geo_df.drop_duplicates(subset=['École', 'Latitude', 'Longitude'])
    
    for _, school in schools_data.iterrows():
        if pd.notna(school['Latitude']) and pd.notna(school['Longitude']):
            folium.CircleMarker(
                location=[school['Latitude'], school['Longitude']],
                radius=5,
                popup=f"{school['École']} ({school['Région']})",
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.7
            ).add_to(school_group)
    
    # Ajouter les labels des régions avec le nombre de questionnaires
    for _, region in region_data.iterrows():
        folium.Popup(
            f"{region['Région']}<br>Questionnaires: {region['count']}",
            parse_html=True
        ).add_to(folium.CircleMarker(
            location=[region['Latitude'], region['Longitude']],
            radius=8,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.9
        ).add_to(m))
    
    # Ajouter les couches à la carte
    m.add_child(NIL)
    m.add_child(school_group)
    m.add_child(colormap)
    
    # Ajouter le contrôle des couches
    folium.LayerControl().add_to(m)
    
    return m

# Créer la carte
carte = make_school_map(geo_df)
# Afficher la carte
carte

In [1]:
a=[f"rgb({255*(19-i)/19}, 0 , {255*i/19}" for i in range(18)]

In [2]:
a

['rgb(255.0, 0 , 0.0',
 'rgb(241.57894736842104, 0 , 13.421052631578947',
 'rgb(228.1578947368421, 0 , 26.842105263157894',
 'rgb(214.73684210526315, 0 , 40.26315789473684',
 'rgb(201.31578947368422, 0 , 53.68421052631579',
 'rgb(187.89473684210526, 0 , 67.10526315789474',
 'rgb(174.47368421052633, 0 , 80.52631578947368',
 'rgb(161.05263157894737, 0 , 93.94736842105263',
 'rgb(147.6315789473684, 0 , 107.36842105263158',
 'rgb(134.21052631578948, 0 , 120.78947368421052',
 'rgb(120.78947368421052, 0 , 134.21052631578948',
 'rgb(107.36842105263158, 0 , 147.6315789473684',
 'rgb(93.94736842105263, 0 , 161.05263157894737',
 'rgb(80.52631578947368, 0 , 174.47368421052633',
 'rgb(67.10526315789474, 0 , 187.89473684210526',
 'rgb(53.68421052631579, 0 , 201.31578947368422',
 'rgb(40.26315789473684, 0 , 214.73684210526315',
 'rgb(26.842105263157894, 0 , 228.1578947368421']

In [1]:
import pandas as pd
import numpy as np
data=pd.read_excel("Data_Simulation.xlsx", sheet_name="Data")
data.columns

Index(['ID_Enquêteur', 'ID_Contrôleur', 'ID_Superviseur', 'Résultat', 'Raison',
       'École', 'Région', 'Latitude', 'Longitude', 'Type_Questionnaire',
       'Date_Collecte'],
      dtype='object')

In [2]:
# Calculer le nombre de questionnaires par école et type
school_counts = data.groupby(['École', 'Type_Questionnaire']).size().unstack(fill_value=0)

# Renommer les colonnes pour plus de clarté
school_counts = school_counts.rename(columns={
    'Enfant': 'Total_enfant',
    'Enseignant': 'Total_enseignant'
})

# Fusionner avec le dataframe original
data = data.merge(school_counts, on='École', how='left')

In [3]:
# Calculer le nombre de questionnaires par école et type
Reg_counts = data.groupby(['Région', 'Type_Questionnaire']).size().unstack(fill_value=0)

# Renommer les colonnes pour plus de clarté
Reg_counts = Reg_counts.rename(columns={
    'Enfant': 'Reg_enf',
    'Enseignant': 'Reg_ens'
})

# Fusionner avec le dataframe original
data = data.merge(Reg_counts, on='Région', how='left')

In [ ]:
data['is_enfant'] = np.where(data['Type_Questionnaire'] == 'Enfant', 1, 0)

le fichier connecxion contient les information comme ceci . j'ai importé dans le fichier DF connect. crée un fichier users.avec celà. et modifie la fonction autehtification pour que ça prènne en paramètre le statut de l'agent ( enquêteur, controleur, superviseur, ou superviseur général. ) de sorte que si on appele la fonction avec un statut et que l'agent qui tente de se connecter n'a pas ce statut, ça lui renvoie un message "Désolé user, vous n'avez pas accès à cet espace, connecter vous à l'espace réservé au "son statut")

In [4]:
data=pd.read_excel("Data_Simulation.xlsx", sheet_name="Data")

In [5]:
data.columns

Index(['ID_Enquêteur', 'ID_Contrôleur', 'ID_Superviseur', 'Résultat', 'Raison',
       'École', 'Région', 'Latitude', 'Longitude', 'Type_Questionnaire',
       'Date_Collecte'],
      dtype='object')

In [6]:
data["ID_Superviseur"].unique()

array(['S_AD', 'S_CE', 'S_ES', 'S_EN', 'S_LI', 'S_NO', 'S_OU', 'S_SU',
       'S_SO'], dtype=object)

In [7]:
# Filter data for Centre region
data_centre = data[data['Région'] == 'Centre']

# Get unique supervisors in Centre region
superviseurs_centre = data_centre[['ID_Superviseur', 'Région']].drop_duplicates()

# Export to Excel
data_centre.to_excel('superviseurs_centre.xlsx', index=False)

# Print summary
print(f"Number of records exported: {len(data_centre)}")
print("\nUnique supervisors in Centre region:")
print(superviseurs_centre)

Number of records exported: 168

Unique supervisors in Centre region:
    ID_Superviseur  Région
126           S_CE  Centre


In [3]:
data.describe()
data.columns


Index(['Établissement', 'ID_Enquêteur', 'ID_Contrôleur', 'ID_Superviseur'], dtype='object')

In [1]:
from ftplib import FTP

In [1]:
from My_Cspro_function import *
import pandas as pd
import numpy as np

In [2]:
dico_eleve=transform_cspro_dict("ELEVE.dcf")
dico_eleve

{'ES00Q00_VS1': {'101011': 'école publique de DARSO',
  '102021': 'école publique de  KANTALONG   (PACARF)',
  '102031': 'école publique de DARA EWA',
  '103041': 'école publique de GASSOL-HARKA',
  '103042': 'école publique de KOUI-LABBARE',
  '104051': 'école publique bilingue de DHOHONG',
  '104061': 'école publique de FADA',
  '105071': 'école publiques  de BEKA MATARI',
  '105081': 'école publique de KOUBADJE',
  '105091': 'école publique bilingue de DANG (PACARF)',
  '201011': 'école publique de BOTOMO',
  '201012': 'école publique de YAMBAYE',
  '201033': 'école publique de BOTOMO',
  '201034': 'école publique de YAMBAYE',
  '201045': 'école publique de BOTOMO',
  '201046': 'école publique de YAMBAYE',
  '201021': 'école bilingue de GUENG',
  '201022': ' école publique de ZAKAN',
  '401011': ' école publique de BIR-PONDO',
  '401012': ' école publique de MOKOLO 2',
  '401013': ' école publique de NKOLBIKONG 3',
  '401021': ' école publique de FRONTIERE',
  '401022': ' école publ

In [7]:
dico_eleve["ES00Q03_VS1"]

{'10101': 'NGAOUNDAL',
 '10202': 'KONTCHA',
 '10203': 'TIGNERE',
 '10304': 'BANYO',
 '10405': 'DJOHONG',
 '10406': 'MEIGANGA',
 '10507': 'MARTAP',
 '10508': 'NGANHA',
 '10509': 'NGAOUNDERE 3ème',
 '20101': 'BOKITO',
 '20102': 'DEUK',
 '20103': 'BOKITO',
 '20104': 'BOKITO',
 '40101': 'BERTOUA 1er',
 '40102': 'GAROUA-BOULAI',
 '50103': 'BOGO',
 '50204': 'FOTOKOL',
 '50205': 'GOULFEY',
 '50206': 'MAKARY',
 '50307': 'GOBO',
 '50308': 'GUERE',
 '50309': 'MAGA',
 '50310': 'WINA',
 '50311': 'YAGOUA',
 '50412': 'DZIGUILAO',
 '50413': 'KAELE',
 '50414': 'MOUTOURWA',
 '50415': 'TOULOUM',
 '50516': 'KOLOFATA',
 '50517': 'TOKOMBERE',
 '50618': 'BOURRHA',
 '50619': 'KOZA',
 '50620': 'MOZOGO',
 '60101': 'MBANGA',
 '60102': 'NKONGSAMBA 1',
 '80101': 'BARNDAKE',
 '80102': 'BASCHEO',
 '80103': 'GAROUA 3ème',
 '80104': 'TOUROUA',
 '80205': 'FIGUIL',
 '80206': 'GUIDER',
 '90101': 'KUMBO',
 '90202': 'NKAMBE',
 '100101': 'MBOUDA',
 '100202': 'KEKEM',
 '100303': 'PENKA-MICHEL',
 '100304': 'SANTCHOU',
 '1004